In [1]:
library(extrafont)
library(tidyverse)
library(epidemia)
library(arrow)

Registering fonts with R

── Attaching core tidyverse packages ───────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ─────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp




# 1. parameters

In [15]:
T = 60
T0 = 40

## observation parameters

In [16]:
alpha = 0.01
r_Y = 10

$$Y_t = NegBinom( \frac{EY_t}{r_Y + EY_t}, r_Y )$$

$$ EY_t = \alpha\sum_{s=1}^{t-1} I_s \pi_{t-s} $$

## latent parameters

In [17]:
d = 2
beta = c(0, -2.2)

In [18]:
r_I = 100
K = 6.5

$$I_t = NegBinom( \frac{EI_t}{r_I + EI_t}, r_I )$$

$$ EI_t = R(\bar{A}_t, \beta) \sum_{s=1}^{t-1} I_s g_{t-s} $$

$$R(\bar{A}_t, \beta) = \frac{K}{1+\exp(- \beta^\top A_t)}$$

## seed values

In [19]:
mu = log(100)

$$I_{-T_0} = \mu_0$$

# 2. fit

In [20]:
num_sim = 1000

In [21]:
data <- read_feather('data/simulation_randinf_0.feather')

In [22]:
head(data)

sim,date,R,EI,infection,EY,death,intervention
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
0,0,3.25,324.9978,356,0.9560691,0,0
0,1,3.25,340.2458,298,0.9625433,0,0
0,2,3.25,392.1966,409,0.9682832,0,0
0,3,3.25,471.0478,545,0.9736541,1,0
0,4,3.25,581.9010,584,0.9797520,4,0
0,5,3.25,726.7304,890,0.9880898,1,0


In [23]:
rt <- epirt(formula = R(country, date) ~ 1 + intervention,
            prior = shifted_gamma(shape = 1/6, scale = 1, shift = log(1.05)/6),
            prior_covariance = decov(shape = c(2, rep(0.5, 5)), scale = 0.25),
            link = scaled_logit(6.5))

In [24]:
inf <- epiinf(gen = EuropeCovid$si, seed_days = T0)

In [25]:
deaths <- epiobs(formula = death ~ 1, i2o = EuropeCovid2$inf2death,
                 prior_intercept = normal(0, 0.2), link = scaled_logit(0.02))

Warning message:
“i2o does not sum to one. Please ensure this is intentional.”


In [26]:
results = array(0, c(num_sim, 6))
colnames(results)=c('b[0]','b[1]','V[0,0]','V[1,0]','V[0,1]','V[1,1]')

In [27]:
for(iter_sim in 1:num_sim){
    start_sim = proc.time()[3]
    
    data_raw = data %>% filter(sim == iter_sim-1)
    data_i = data.frame(country = factor(rep(1,T+T0)),
                        date = as.Date("2022-01-01")
                               + c(-T0:-1, data_raw$date),
                        death = c(rep(NA,T0), data_raw$death),
                        intervention = c(rep(0,T0), data_raw$intervention))
    
    args <- list(rt = rt, inf = inf, obs = deaths, data = data_i, 
                 seed=12345, refresh=0)
    args$algorithm <- "fullrank"; args$iter <- 5e4; args$tol_rel_obj <- 1e-3
    
    fm <- do.call(epim, args)
    
    results[iter_sim,1:2] = colMeans(as.data.frame(fm$stanfit)[1:2])
    results[iter_sim,3:6] = cov(as.data.frame(fm$stanfit)[1:2])
    
    print(paste0(iter_sim,"-th simulation finished, ",
                 proc.time()[3] - start_sim," sec."))
    flush(stdout())
}

[1] "1-th simulation finished, 5.354 sec."
[1] "2-th simulation finished, 5.45399999999999 sec."
[1] "3-th simulation finished, 5.443 sec."
[1] "4-th simulation finished, 5.24 sec."
[1] "5-th simulation finished, 5.274 sec."
[1] "6-th simulation finished, 5.274 sec."
[1] "7-th simulation finished, 5.35599999999999 sec."
[1] "8-th simulation finished, 5.217 sec."
[1] "9-th simulation finished, 5.31100000000001 sec."
[1] "10-th simulation finished, 5.196 sec."
[1] "11-th simulation finished, 5.27700000000002 sec."
[1] "12-th simulation finished, 5.19399999999999 sec."
[1] "13-th simulation finished, 5.28100000000001 sec."
[1] "14-th simulation finished, 5.19 sec."
[1] "15-th simulation finished, 5.22 sec."
[1] "16-th simulation finished, 5.27000000000001 sec."
[1] "17-th simulation finished, 5.30800000000002 sec."
[1] "18-th simulation finished, 5.21799999999999 sec."
[1] "19-th simulation finished, 5.22 sec."
[1] "20-th simulation finished, 5.25800000000001 sec."
[1] "21-th simulation f

[1] "154-th simulation finished, 5.45299999999997 sec."
[1] "155-th simulation finished, 5.44899999999996 sec."
[1] "156-th simulation finished, 5.43600000000004 sec."
[1] "157-th simulation finished, 5.56799999999998 sec."
[1] "158-th simulation finished, 5.45100000000002 sec."
[1] "159-th simulation finished, 5.58000000000004 sec."
[1] "160-th simulation finished, 5.45699999999999 sec."
[1] "161-th simulation finished, 5.44500000000005 sec."
[1] "162-th simulation finished, 5.44099999999992 sec."
[1] "163-th simulation finished, 5.50800000000004 sec."
[1] "164-th simulation finished, 5.55799999999999 sec."
[1] "165-th simulation finished, 5.452 sec."
[1] "166-th simulation finished, 5.43999999999994 sec."
[1] "167-th simulation finished, 5.43600000000004 sec."
[1] "168-th simulation finished, 5.553 sec."
[1] "169-th simulation finished, 5.43900000000008 sec."
[1] "170-th simulation finished, 5.40899999999999 sec."
[1] "171-th simulation finished, 5.43100000000004 sec."
[1] "172-th si

[1] "302-th simulation finished, 5.53899999999999 sec."
[1] "303-th simulation finished, 5.55999999999995 sec."
[1] "304-th simulation finished, 5.65300000000002 sec."
[1] "305-th simulation finished, 5.53199999999993 sec."
[1] "306-th simulation finished, 5.55799999999999 sec."
[1] "307-th simulation finished, 5.51700000000005 sec."
[1] "308-th simulation finished, 5.6110000000001 sec."
[1] "309-th simulation finished, 5.47000000000003 sec."
[1] "310-th simulation finished, 5.52600000000007 sec."
[1] "311-th simulation finished, 5.5 sec."
[1] "312-th simulation finished, 5.55500000000006 sec."
[1] "313-th simulation finished, 5.77499999999986 sec."
[1] "314-th simulation finished, 5.57500000000005 sec."
[1] "315-th simulation finished, 5.53500000000008 sec."
[1] "316-th simulation finished, 5.548 sec."
[1] "317-th simulation finished, 5.56600000000003 sec."
[1] "318-th simulation finished, 5.55899999999997 sec."
[1] "319-th simulation finished, 5.5 sec."
[1] "320-th simulation finishe

Warning message:
“Pareto k diagnostic value is 0.74. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "409-th simulation finished, 5.63200000000006 sec."
[1] "410-th simulation finished, 5.41499999999996 sec."
[1] "411-th simulation finished, 5.44300000000021 sec."
[1] "412-th simulation finished, 5.51099999999997 sec."
[1] "413-th simulation finished, 5.60199999999986 sec."
[1] "414-th simulation finished, 5.46299999999974 sec."
[1] "415-th simulation finished, 5.44800000000032 sec."
[1] "416-th simulation finished, 5.43000000000029 sec."
[1] "417-th simulation finished, 5.55299999999988 sec."
[1] "418-th simulation finished, 5.5630000000001 sec."
[1] "419-th simulation finished, 5.49500000000035 sec."
[1] "420-th simulation finished, 5.43899999999985 sec."
[1] "421-th simulation finished, 5.49600000000009 sec."
[1] "422-th simulation finished, 5.61499999999978 sec."
[1] "423-th simulation finished, 5.51199999999972 sec."
[1] "424-th simulation finished, 5.48199999999997 sec."
[1] "425-th simulation finished, 5.46299999999974 sec."
[1] "426-th simulation finished, 5.57000000000016

[1] "556-th simulation finished, 5.50999999999976 sec."
[1] "557-th simulation finished, 5.51299999999992 sec."
[1] "558-th simulation finished, 5.63900000000012 sec."
[1] "559-th simulation finished, 5.47800000000007 sec."
[1] "560-th simulation finished, 5.50099999999975 sec."
[1] "561-th simulation finished, 5.51899999999978 sec."
[1] "562-th simulation finished, 5.6239999999998 sec."
[1] "563-th simulation finished, 5.47600000000011 sec."
[1] "564-th simulation finished, 5.49299999999994 sec."
[1] "565-th simulation finished, 5.50900000000001 sec."
[1] "566-th simulation finished, 5.50500000000011 sec."
[1] "567-th simulation finished, 5.55099999999993 sec."
[1] "568-th simulation finished, 5.60899999999992 sec."
[1] "569-th simulation finished, 5.51599999999962 sec."
[1] "570-th simulation finished, 5.57900000000018 sec."
[1] "571-th simulation finished, 5.5300000000002 sec."
[1] "572-th simulation finished, 5.51999999999998 sec."
[1] "573-th simulation finished, 5.50199999999995 

[1] "703-th simulation finished, 5.0300000000002 sec."
[1] "704-th simulation finished, 5.11299999999983 sec."
[1] "705-th simulation finished, 5.04500000000007 sec."
[1] "706-th simulation finished, 5.07200000000012 sec."
[1] "707-th simulation finished, 5.07200000000012 sec."
[1] "708-th simulation finished, 5.18000000000029 sec."
[1] "709-th simulation finished, 5.05700000000024 sec."
[1] "710-th simulation finished, 5.13000000000011 sec."
[1] "711-th simulation finished, 5.12800000000016 sec."
[1] "712-th simulation finished, 5.13599999999997 sec."
[1] "713-th simulation finished, 5.11400000000003 sec."
[1] "714-th simulation finished, 5.04899999999998 sec."
[1] "715-th simulation finished, 5.096 sec."
[1] "716-th simulation finished, 5.08800000000019 sec."
[1] "717-th simulation finished, 5.20899999999983 sec."
[1] "718-th simulation finished, 5.07499999999982 sec."
[1] "719-th simulation finished, 5.12899999999991 sec."
[1] "720-th simulation finished, 5.07600000000002 sec."
[1] 

[1] "850-th simulation finished, 5.10300000000007 sec."
[1] "851-th simulation finished, 5.04100000000017 sec."
[1] "852-th simulation finished, 5.10899999999947 sec."
[1] "853-th simulation finished, 5.18199999999979 sec."
[1] "854-th simulation finished, 5.11499999999978 sec."
[1] "855-th simulation finished, 5.07899999999972 sec."
[1] "856-th simulation finished, 5.08200000000033 sec."
[1] "857-th simulation finished, 5.07999999999993 sec."
[1] "858-th simulation finished, 5.08100000000013 sec."
[1] "859-th simulation finished, 5.03000000000065 sec."
[1] "860-th simulation finished, 5.06900000000041 sec."
[1] "861-th simulation finished, 5.16799999999967 sec."
[1] "862-th simulation finished, 5.0630000000001 sec."
[1] "863-th simulation finished, 5.05599999999959 sec."
[1] "864-th simulation finished, 5.10500000000047 sec."
[1] "865-th simulation finished, 5.01199999999972 sec."
[1] "866-th simulation finished, 5.04899999999998 sec."
[1] "867-th simulation finished, 5.20100000000002

[1] "997-th simulation finished, 5.14800000000014 sec."
[1] "998-th simulation finished, 5.0619999999999 sec."
[1] "999-th simulation finished, 5.11300000000028 sec."
[1] "1000-th simulation finished, 5.15899999999965 sec."


In [28]:
write_feather(as.data.frame(results), 
              'coverage_epidemia_nbinom_randinf_0.feather')